# Exploring Iris to remember the structure firstly


In [7]:
import sys

print(sys.executable)

w:\7_Semester\Machine Learning\labs\.venv\Scripts\python.exe


In [1]:
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris.data, iris.target

print(f"rows in the dataset: {X.shape[0]}")
print("==========================================")
print(f"number of features: {X.shape[1]}")
print("==========================================")
print(f"labels of the columns: {iris.feature_names}")
print("==========================================")
print(f"Labels: {iris.target_names}")

rows in the dataset: 150
number of features: 4
labels of the columns: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Labels: ['setosa' 'versicolor' 'virginica']


In [2]:
print("==========================================")
print(f"Feature matrix (X): \n{X[:5]}")

Feature matrix (X): 
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


In [3]:
print(f"Target vector (y): {y}")

Target vector (y): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


## What info did I get from that?

- input shape needs to be given in the input layer, it needs to be 4 (since iris dataset got 4 columns)
- 3 labels, that means we gotta use softmax with 3 neurons in the output layer instead of a sigmoid

---


## **1. Simple ANN on Iris dataset with a softmax activation and diff activations comparison at the hidden layer**


In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def run_multi_class_classification(hidden_activation):
    iris = load_iris()
    X, y = iris.data, iris.target
    y = keras.utils.to_categorical(y, num_classes=3)

    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1],)),
        layers.Dense(64, activation=hidden_activation),
        layers.Dense(3, activation='softmax'),
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=50, verbose=0)
    return model.evaluate(X_test, y_test, verbose=0)[1]

activation_functions = {
    'Tanh': 'tanh',
    'Sigmoid': 'sigmoid',
    'ReLU': 'relu',
    'Leaky ReLU': layers.LeakyReLU(alpha=0.2),
    'Linear': 'linear',
}

print("=== Multi-Class Classification with Different Hidden Layer Activations on Iris Dataset ===")
for name, activation in activation_functions.items():
    acc = run_multi_class_classification(activation)
    print(f'Accuracy with {name} activation (Softmax output): {acc:.4f}')


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
import random
import itertools
import optuna
import keras_tuner as kt
import pyswarms as ps  # not used below, but included per pdf

# data
wine = load_wine()
X = StandardScaler().fit_transform(wine.data)
y = wine.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model factory
def build_model(optimizer='adam', activation='relu', neurons=32):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X.shape[1], activation=activation))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

optimizers = ['adam', 'sgd', 'rmsprop']
activations = ['relu', 'sigmoid']
neurons_list = [16, 32, 64]
batch_sizes = [16, 32, 64]
epochs_list = [10, 20, 30]

# random search
def random_search(n_iter=5):
    best_acc, best_params = 0, None
    for _ in range(n_iter):
        params = {
            'optimizer': random.choice(optimizers),
            'activation': random.choice(activations),
            'neurons': random.choice(neurons_list),
            'batch_size': random.choice(batch_sizes),
            'epochs': random.choice(epochs_list),
        }
        model = build_model(params['optimizer'], params['activation'], params['neurons'])
        model.fit(X_train, y_train, batch_size=params['batch_size'], epochs=params['epochs'], verbose=0)
        y_pred = np.argmax(model.predict(X_test), axis=1)
        acc = accuracy_score(y_test, y_pred)
        if acc > best_acc:
            best_acc, best_params = acc, params
    return best_params

# grid search
def grid_search():
    best_acc, best_params = 0, None
    for combo in itertools.product(optimizers, activations, neurons_list, batch_sizes, epochs_list):
        params = {
            'optimizer': combo[0],
            'activation': combo[1],
            'neurons': combo[2],
            'batch_size': combo[3],
            'epochs': combo[4],
        }
        model = build_model(params['optimizer'], params['activation'], params['neurons'])
        model.fit(X_train, y_train, batch_size=params['batch_size'], epochs=params['epochs'], verbose=0)
        y_pred = np.argmax(model.predict(X_test), axis=1)
        acc = accuracy_score(y_test, y_pred)
        if acc > best_acc:
            best_acc, best_params = acc, params
    return best_params

# hyperband (keras tuner)
def hyperband_model(hp):
    model = Sequential()
    model.add(Dense(hp.Choice('neurons', neurons_list),
                    activation=hp.Choice('activation', activations),
                    input_shape=(X.shape[1],)))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=hp.Choice('optimizer', optimizers),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.Hyperband(
    hyperband_model,
    objective='val_accuracy',
    max_epochs=30,
    directory='hyperband_dir',
    project_name='wine'
)
tuner.search(X_train, y_train, validation_split=0.2, verbose=0)
best_hyperband = tuner.get_best_hyperparameters(1)[0].values

# bayesian (optuna)
def objective(trial):
    params = {
        'optimizer': trial.suggest_categorical('optimizer', optimizers),
        'activation': trial.suggest_categorical('activation', activations),
        'neurons': trial.suggest_categorical('neurons', neurons_list),
        'batch_size': trial.suggest_categorical('batch_size', batch_sizes),
        'epochs': trial.suggest_categorical('epochs', epochs_list),
    }
    model = build_model(params['optimizer'], params['activation'], params['neurons'])
    model.fit(X_train, y_train, batch_size=params['batch_size'], epochs=params['epochs'], verbose=0)
    y_pred = np.argmax(model.predict(X_test), axis=1)
    return accuracy_score(y_test, y_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)
best_bayes = study.best_params

# run & report
best_random = random_search()
best_grid = grid_search()

print("Best Random Search Params:", best_random)
print("Best Grid Search Params:", best_grid)
print("Best Hyperband Params:", best_hyperband)
print("Best Bayesian Params:", best_bayes)
